In [48]:
import pandas as pd
import numpy as np
import sqlite3
import importlib
from datetime import datetime

# keeping company information in additional file
import data_file

In [49]:
sales_person_n = data_file.sales_person_n
exlbu=data_file.exlbu
exlpanrter=data_file.exlpanrter

# Get current year and month
now = datetime.now()
last_month = now
# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

In [50]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
conn.close()

conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [ ]:
df_orders['sold_to_customer'] = df_orders['sold_to_customer'].astype(str)
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
df_orders[~df_orders['sold_to_customer'].isin(df_customers['sold_to_customer'].to_list())]
#print('order data lines:', len(df_orders), "Total Order Intake Amount: {:.2f}".format(df_orders.order_intake_amount_eur.sum()))

In [182]:
# for testing purposes I keep this lines
wdf = df_customers.copy()
wdf.loc[wdf['tier'].isna(), 'tier'] = 'Direct'
wdf.loc[wdf['tier'] == 'None', 'tier'] = 'Direct'
wdf.loc[wdf['tier_new'].isna(), 'tier_new'] = 'Direct'
wdf.loc[wdf['tier_new'] == 'None', 'tier_new'] = 'Direct'
wdf.loc[wdf[exlpanrter] == 'None', exlpanrter] = 'NA'
wdf.loc[wdf[exlpanrter].isna(), exlpanrter] = 'NA'

dfc = df_orders.copy()

In [183]:
bu_defin = pd.read_excel('data_files/bu_defin.xlsx')
bu_defin = bu_defin.loc[:, 'bu2':]
bu_defin['for_bu'] = bu_defin['for_bu'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].str.strip()

In [184]:
order_data = dfc.loc[:,['company_code_n', 'year_month', 'FY', 
       'bu2', 'sales_person_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n', 'ec_eu_customer_n', 'order_intake_amount_eur']]

order_data['bu2'] = order_data['bu2'].astype(str)
order_data['bu2'] = order_data['bu2'].str.strip()

# add bu column for future reports
order_data = order_data.merge(bu_defin, how='left' )

In [185]:
# Convert 'FY' column to datetime format
order_data['FY'] = pd.to_datetime(order_data['FY'])

In [186]:
order_columns = list(order_data.columns) + ['customer_name', 'indirect_direct', 'tier', 'tier_new', 'type', exlpanrter]

In [187]:
# special case with specific SO for agent
agent_so = data_file.special_so
agent_so_two = data_file.special_so_two 
agent_one = data_file.agent_one
so_order_data = order_data[(order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two)]
order_data = order_data[~((order_data['sales_order_so'] == agent_so) | (order_data['sales_order_so'] == agent_so_two))]
print('should be 0: ', len(order_data) + len(so_order_data) - len(df_orders))

so_order_data['customer_name'] = agent_one
so_order_data['indirect_direct'] = 'Indirect'
so_order_data['tier'] = 'Channel Partner'
so_order_data['type'] = 'Agent'

should be 0:  0


In [188]:
# special case with specific cutomers for agent
partners_data = wdf.copy()
agent_two = data_file.agent_two
partners_data.loc[partners_data[exlpanrter].notna(), 'customer_name'] = agent_two
partners_data = partners_data.loc[:, ['sold_to_customer', 'agent_person', 'company_code_n', 'sold_to_customer_n','customer_name', 
       'indirect_direct', 'channel', 'type', 'tier', 'tier_new',  exlpanrter]]

partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)

In [189]:
agent_partners = partners_data[partners_data['agent_person'].notna()]
agent_partners = agent_partners[agent_partners['agent_person'] != 'None']

# agent sales records preparation
agents_order_data = order_data[order_data['sales_person_n'].isin(sales_person_n)]
order_data = order_data[~order_data['sales_person_n'].isin(sales_person_n)]

agents_order_before = agents_order_data.copy()

print('should be 0: ', len(agents_order_data) + len(order_data) +  len(so_order_data) - len(df_orders))

agents_order_data = agents_order_data.merge(agent_partners, left_on='sales_person_n', right_on='sold_to_customer_n', how='left')
agents_order_data = agents_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
agents_order_data.columns = agents_order_data.columns.str.replace('_x', '')
agents_order_data = agents_order_data.loc[:, order_columns]
agents_order_data['type'] = 'Agent'

print('should be 0: ', len(agents_order_data) + len(order_data) +  len(so_order_data) - len(df_orders))

should be 0:  0
should be 0:  0


In [190]:
# all partners except agents data preparation
other_order_data = order_data[order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
checkthis = order_data[~order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
print('not in the customer db:', len(checkthis))
other_order_data = order_data.copy()
other_order_data = other_order_data.merge(partners_data, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
other_order_data = other_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
other_order_data.columns = other_order_data.columns.str.replace('_x', '')
other_order_data = other_order_data.loc[:, order_columns]
# Update values in columns based on the specific value
other_order_data.loc[((other_order_data['indirect_direct'] == 'Direct') & ( (other_order_data['tier_new'] != 'Direct') | (other_order_data[exlpanrter] != 'NA' ) )), 'tier'] = 'Channel Partner'
other_order_data.loc[((other_order_data['indirect_direct'] == 'Direct') & ( (other_order_data['tier_new'] != 'Direct') | (other_order_data[exlpanrter] != 'NA') )), 'tier_new'] = 'Channel Partner'
other_order_data.loc[((other_order_data['indirect_direct'] == 'Direct') & ( (other_order_data['tier_new'] != 'Direct') | (other_order_data[exlpanrter] != 'NA') )), 'type'] = 'Agent' 
print('should be 0: ', len(agents_order_data) + len(other_order_data) +  len(so_order_data) - len(df_orders))

not in the customer db: 1
should be 0:  0


In [ ]:
checkthis

In [192]:
# combining all the sales records
full_data = pd.concat([other_order_data, agents_order_data, so_order_data])
full_data.reset_index(inplace=True, drop=True)
#full_data.drop_duplicates(inplace=True)
print('should be 0: ', len(full_data) - len(df_orders))

should be 0:  0


In [193]:
# Create a dictionary mapping sold_to_customer values to corresponding countries
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
customer_country_map = df_customers.set_index('sold_to_customer')['countries'].to_dict()

# Fill missing values in temp_df['countries'] using the mapping dictionary
full_data['countries'] = ''
full_data['sold_to_customer'] = full_data['sold_to_customer'].astype(str)
full_data['countries'] = full_data['countries'].replace('', pd.NA).fillna(full_data['sold_to_customer'].map(customer_country_map))

checkthis = full_data[full_data.countries.isna()]

In [ ]:
checkthis

In [195]:
# Drop rows where 'countries' column contains NaN values
full_data = full_data.dropna(subset=['countries'])

In [ ]:
full_data.order_intake_amount_eur.sum() - dfc.order_intake_amount_eur.sum() + checkthis.order_intake_amount_eur.sum()

In [181]:
full_data = full_data[['company_code_n', 'year_month', 'FY', 'bu2', 'sales_person_n',
       'sales_order_so', 'sold_to_customer', 'sold_to_customer_n',
       'ec_eu_customer_n', 'order_intake_amount_eur', 'for_bu',
       'customer_name', 'indirect_direct', 'tier', 'tier_new', 'type',
       'countries']]

writer = pd.ExcelWriter(f'data_files/outcome/full_order_data_set_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
full_data.to_excel(writer, sheet_name='results', index=False)
# Save the file
writer.close()